In [ ]:
!pip install opencv-contrib-python==4.4.0.44
!pip install -U scikit-learn
!pip install joblib

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import argparse as ap
import cv2
import imutils 
import numpy as np
import os
import glob
import math
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
#from sklearn.externals import joblib
import joblib
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler

import time
from IPython.display import display, clear_output

# Train Model & Find Features


In [ ]:
train_path = "Dataset/train/"
training_names = os.listdir(train_path)
print(training_names)

['book', 'cup', 'box']


In [ ]:
image_books = glob.glob(train_path+"book/*")
image_cups = glob.glob(train_path+"cup/*")
image_boxes = glob.glob(train_path+"box/*")
print("Total Book Observations:\t", len(image_books))
print("Total Cups Observations:\t", len(image_cups))
print("Total Boxes Observations:\t", len(image_boxes))


Total Book Observations:	 280
Total Cups Observations:	 257
Total Boxes Observations:	 265


In [ ]:
# Get all the path to the images and save them in a list
# image_paths and the corresponding label in image_paths
image_paths_train = []
image_classes = []
class_id = 0
for training_name in training_names:
    dir = os.path.join(train_path, training_name)
    #print(dir)
    class_path = imutils.imlist(dir)
    #print(class_path)
    image_paths_train+=class_path
    #print(image_paths_train)
    image_classes+=[class_id]*len(class_path)
    #print(image_classes)
    class_id+=1

In [ ]:
# Create feature extraction and keypoint detector objects
#mser = cv2.MSER_create()
#mser = cv2.FeatureDetector_create('MSER')
sift = cv2.xfeatures2d.SIFT_create()
#surf = cv2.xfeatures2d.SURF_create()
#fea_det = cv2.FeatureDetector_create("SIFT")
#des_ext = cv2.DescriptorExtractor_create("SIFT")

# List where all the descriptors are stored
des_list = []
i = 1
print("Extracting features")
#i=0
for image_path in image_paths_train:
    im = cv2.imread(image_path)
    im = cv2.resize(im,(256,256))
    #im.resize((512,512))
    #if(image_path.find("book") > 0):
    im = cv2.GaussianBlur(im, (3,3), 20)
    #plt.imshow(im_original)
    #plt.show()
    #plt.imshow(im)
    #plt.show()
    #im = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    kpts, des = sift.detectAndCompute(im, None)
    #kpts, des = sift.detectAndCompute(im, None)
    #kpts = fea_det.detect(im)
    #kpts, des = des_ext.compute(im, kpts)
    des_list.append((image_path, des))  
    #imK = im.copy()
    #imK = cv2.drawKeypoints(im,kpts, imK) 
    #print(image_path)
    #plt.imshow(imK)
    #plt.show()
    clear_output(wait=True)
    display('Image '+str(i)+' over : '+str(len(image_paths_train)))
    #time.sleep(1)
    i += 1
    #i+=1
    #if(i>3):
    #    break
    
joblib.dump((des_list), "descriptors.pkl", compress=3)

'Image 802 over : 802'

['descriptors.pkl']

In [ ]:
des_list = joblib.load("descriptors.pkl")
# Stack all the descriptors vertically in a numpy array
descriptors = des_list[0][1]
image_classes = []
i = 0
for image_path, descriptor in des_list[1:]:
    try:
        descriptors = np.vstack((descriptors, descriptor))

        if(image_path.find("/book/") > 0):
            image_classes.append(0) # book - 0
        elif(image_path.find("/box/") > 0):
            image_classes.append(2) # box - 2
        else:
            image_classes.append(1) # cup - 1

    except ValueError:
        #print(image_path)
        i+=1
        continue
print("outlier %s on %s"%(i,np.shape(des_list)[0]))
#print(descriptors)

image_classes = np.asarray(image_classes)
print(image_classes.shape)
#print(image_classes)

outlier 0 on 802
(801,)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# Perform k-means clustering
print("k-means clustering")
k = 200
voc, variance = kmeans(descriptors, k, 50, 0.001) 

k-means clustering


In [ ]:
# Calculate the histogram of features
print("Calculate histogram of features")
im_features = np.zeros((len(image_classes), k), "float32")
for i in range(len(image_classes)):
    try:
        words, distance = vq(des_list[i][1],voc)
        for w in words:
            im_features[i][w] += 1
    except ValueError:
        continue
    

# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (im_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_classes)+1) / (1.0*nbr_occurences + 1)), 'float32')

# Scaling the words
stdSlr = StandardScaler().fit(im_features)
im_features = stdSlr.transform(im_features)
print(np.shape(des_list))
print(im_features.shape)
#print(np.array(image_classes))

Calculate histogram of features
(802, 2)
(801, 200)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
# Train the Linear SVM
print("Train SVM")
#svc = SVC()
#clf = GridSearchCV(svc, param_grid)
#clf = LinearSVC(tol=1e-10,max_iter=1e6)
#clf = LinearSVC()
#clf = SVC( C=6, kernel='rbf', gamma='scale', shrinking=True, tol=1e-6, cache_size=200, decision_function_shape='ovr') # -------- test set 4
#clf = SVC( C=1000, kernel='linear', tol=1e-6, cache_size=200, decision_function_shape='ovr')

clf = SVC( C=3, kernel='rbf', gamma='scale', shrinking=True, tol=1e-6, cache_size=200, decision_function_shape='ovr') # -------- test set occluded

clf.fit(im_features, np.array(image_classes))
#clf.fit(train_features, np.array(image_classes))
#sorted(clf.cv_results_.keys())
# Save the SVM
print("Save SVM")
joblib.dump((clf, training_names, stdSlr, k, voc, im_features), "bof.pkl", compress=3)
# save olso in picke format 
import pickle
save=open('bof_p.pkl', 'wb')
obj = pickle.dump((clf, training_names, stdSlr, k, voc, im_features),save,3)

Train SVM
Save SVM


['bof.pkl']

# Test Model


In [ ]:
# Load the classifier, class names, scaler, number of clusters and vocabulary 
clf, classes_names, stdSlr, k, voc, train_features = joblib.load("bof.pkl")


# Get the path of the testing image(s) and store them in a list
image_paths = []
test_path = './Dataset/test_unoccluded/' 
try:
    testing_names = os.listdir(test_path)
except OSError:
    print("No such directory %s\nCheck if the file exists"%(test_path))
    exit()
for testing_name in testing_names:
    dir = os.path.join(test_path, testing_name)
    class_path = imutils.imlist(dir)
    image_paths+=class_path

In [ ]:
# Create feature extraction and keypoint detector objects
sift = cv2.xfeatures2d.SIFT_create(200)
#fea_det = cv2.FeatureDetector_create("SIFT")
#des_ext = cv2.DescriptorExtractor_create("SIFT")

# List where all the descriptors are stored
des_list_test = []

for image_path in image_paths:
    im = cv2.imread(image_path)
    #im = cv2.resize(im,(256,256))
    #im = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    #im = cv2.GaussianBlur(im_original, (47,47), 1000)
    #im = cv2.GaussianBlur(im, (5,5), 10)
    #im = cv2.GaussianBlur(im, (3,3), 20)
    if (im == None).any():
        print("No such file %s\nCheck if the file exists"%(image_path))
        exit()
    #kpts = fea_det.detect(im)
    #kpts, des = des_ext.compute(im, kpts)
    kpts, des = sift.detectAndCompute(im,None)
    des_list_test.append((image_path, des))   
    
    #imK = im.copy()
    #imK = cv2.drawKeypoints(im,kpts, imK) 
    #print(image_path)
    #plt.imshow(imK)
    #plt.show()
    #print(des.shape)

print("Save test features")
joblib.dump((des_list_test), "des_list_test.pkl", compress=3)

Save test features


['des_list_test.pkl']

In [ ]:
des_list_test = joblib.load("des_list_test.pkl")
# Stack all the descriptors vertically in a numpy array
descriptors = des_list_test[0][1]
for image_path, descriptor in des_list_test[0:]:
    descriptors = np.vstack((descriptors, descriptor)) 

# 
test_features = np.zeros((len(image_paths), k), "float32")
for i in range(len(image_paths)):
    words, distance = vq(des_list_test[i][1],voc)
    for w in words:
        test_features[i][w] += 1

# Perform Tf-Idf vectorization
nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
idf = np.array(np.log((1.0*len(image_paths)+1) / (1.0*nbr_occurences + 1)), 'float32')

# Scale the features
test_features = stdSlr.transform(test_features)
print("Save test features")
joblib.dump((test_features), "test_feat.pkl", compress=3)

Save test features


['test_feat.pkl']

#### Prediction with SVM

In [ ]:
# Perform the predictions
#print(clf.predict(test_features))
test_features = joblib.load("test_feat.pkl")
predictions =  [classes_names[i] for i in clf.predict(test_features)]
visulaize = True
res = {"book": 0,"box": 0,"cup": 0}
# Visualize the results, if "visualize" flag set to true by the user
if visulaize:
    for image_path, prediction in zip(image_paths, predictions):
        #image = cv2.imread(image_path)
        print("image: %s - label: %s"%(image_path, prediction))
        if(image_path.find(prediction) > 0):
            res[prediction]+=1
        #cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        #pt = (0, 3 * image.shape[0] // 4)
        #cv2.putText(image, prediction, pt ,cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 2, [0, 255, 0], 2)
        #cv2.imshow("Image", image)
        #cv2.waitKey(50)
print("Reuslts out of  %g test images"%(len(predictions)))
print(res)
ris = [res[k] for k in res]
print("Accuracy: %.3g"%(sum(ris)/len(predictions)))
#cv2.destroyAllWindows()

image: 5/book/left_occ_obj_000577.png - label: book
image: 5/book/left_occ_obj_000576.png - label: book
image: 5/book/left_occ_obj_000580.png - label: book
image: 5/book/left_occ_obj_000578.png - label: book
image: 5/book/left_occ_obj_000581.png - label: book
image: 5/book/left_occ_obj_000579.png - label: book
image: 5/book/left_occ_obj_000582.png - label: book
image: 5/book/left_occ_obj_000585.png - label: book
image: 5/book/left_occ_obj_000584.png - label: book
image: 5/book/left_occ_obj_000583.png - label: book
image: 5/book/left_occ_obj_000588.png - label: book
image: 5/book/left_occ_obj_000586.png - label: book
image: 5/book/left_occ_obj_000587.png - label: book
image: 5/book/left_occ_obj_000589.png - label: book
image: 5/book/left_occ_obj_000680.png - label: cup
image: 5/book/left_occ_obj_000681.png - label: box
image: 5/book/left_occ_obj_000679.png - label: box
image: 5/book/left_occ_obj_000682.png - label: box
image: 5/book/left_occ_obj_000684.png - label: box
image: 5/book/lef

#### Nearest neighbour

In [ ]:
predictions = [None]*(len(test_features))

for j,test in enumerate(test_features):
    min = 1e100
    for i, train in enumerate(train_features):
        val = math.sqrt(sum(pow(test-train,2)))
        if(val < min):
            min = val;
            #print(image_classes[i])
            #print(classes_names[0])
            predictions[j] = classes_names[image_classes[i]]
res = {"book": 0,"box": 0,"cup": 0}
# Visualize the results, if "visualize" flag set to true by the user
if visulaize:
    for image_path, prediction in zip(image_paths, predictions):
        #image = cv2.imread(image_path)
        print("image: %s - label: %s"%(image_path, prediction))
        if(image_path.find(prediction) > 0):
            res[prediction]+=1
        #cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        #pt = (0, 3 * image.shape[0] // 4)
        #cv2.putText(image, prediction, pt ,cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 2, [0, 255, 0], 2)
        #cv2.imshow("Image", image)
        #cv2.waitKey(50)
print("Reuslts out of  %g test images"%(len(predictions)))
print(res)
ris = [res[k] for k in res]
print("Accuracy: %.3g"%(sum(ris)/len(predictions)))

['cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'box', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'box', 'book', 'cup', 'cup', 'cup', 'book', 'book', 'book', 'cup', 'book', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'book', 'box', 'cup', 'cup', 'box', 'cup', 'box', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'box', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'box', 'cup', 'cup', 'book', 'box', 'book', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'box', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup', 'cup

### More test

In [ ]:
max = 0
max_c = -1
for i in range(1,50):
    # Train the Linear SVM
    #print("Train SVM")
    #svc = SVC()
    #clf = GridSearchCV(svc, param_grid)
    #clf = LinearSVC(tol=1e-10,max_iter=1e6)
    #clf = LinearSVC()
    c = i
    print("c: "+str(c))
    clf = SVC( C=c, kernel='rbf', gamma='scale', shrinking=True, tol=1e-6, cache_size=200, decision_function_shape='ovr')
    #clf = SVC( C=1000, kernel='linear', tol=1e-6, cache_size=200, decision_function_shape='ovr')
    clf.fit(im_features, np.array(image_classes))
    #sorted(clf.cv_results_.keys())
    # Save the SVM
    #print("Save SVM")
    #joblib.dump((clf, training_names, stdSlr, k, voc, im_features), "bof.pkl", compress=3)


    # Load the classifier, class names, scaler, number of clusters and vocabulary 
    #clf, classes_names, stdSlr, k, voc, train_features = joblib.load("bof.pkl")


    # Get the path of the testing image(s) and store them in a list
    image_paths = []
    #test_path = 'drive/MyDrive/Denmark/DTU/1 Year/II semester /PfAS/Final Project/Dataset/resized_data/test/'
    test_path = './Dataset/test_unoccluded/' 
    try:
        testing_names = os.listdir(test_path)
    except OSError:
        print("No such directory %s\nCheck if the file exists"%(test_path))
        exit()
    for testing_name in testing_names:
        dir = os.path.join(test_path, testing_name)
        class_path = imutils.imlist(dir)
        image_paths+=class_path


    # Perform the predictions
    #print(clf.predict(test_features))
    test_features = joblib.load("test_feat.pkl")
    predictions =  [classes_names[i] for i in clf.predict(test_features)]
    visulaize = True
    res = {"book": 0,"box": 0,"cup": 0}
    # Visualize the results, if "visualize" flag set to true by the user
    if visulaize:
        for image_path, prediction in zip(image_paths, predictions):
            #image = cv2.imread(image_path)
            #print("image: %s - label: %s"%(image_path[97:], prediction))
            if(image_path.find(prediction) > 0):
                res[prediction]+=1
            #cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
            #pt = (0, 3 * image.shape[0] // 4)
            #cv2.putText(image, prediction, pt ,cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 2, [0, 255, 0], 2)
            #cv2.imshow("Image", image)
            #cv2.waitKey(50)
    print("Reuslts out of  %g test images - TRAIN 3 - (365, 250, 250)"%(len(predictions)))
    print(res)
    ris = [res[k] for k in res]
    acc = sum(ris)/len(predictions)
    print("Accuracy: %.3g"%(acc))
    if(acc > max):
        max = acc
        max_c = c
    #cv2.destroyAllWindows()

print("Max acc: %g at C:%d"%(max,max_c))

c: 1
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 76, 'box': 81, 'cup': 28}
Accuracy: 0.663
c: 2
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 76, 'box': 76, 'cup': 40}
Accuracy: 0.688
c: 3
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 76, 'box': 73, 'cup': 55}
Accuracy: 0.731
c: 4
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 76, 'box': 69, 'cup': 66}
Accuracy: 0.756
c: 5
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 76, 'box': 67, 'cup': 70}
Accuracy: 0.763
c: 6
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 73, 'box': 65, 'cup': 68}
Accuracy: 0.738
c: 7
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 72, 'box': 63, 'cup': 68}
Accuracy: 0.728
c: 8
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book': 73, 'box': 63, 'cup': 67}
Accuracy: 0.728
c: 9
Reuslts out of  279 test images - TRAIN 3 - (365, 250, 250)
{'book'

### Single function

In [ ]:
# variables needed: voc, k, class_names, clf 
def classifyBoW(image):
    # load parmeters (can be loaded only ones at the beginning of the code)
    #clf, classes_names, stdSlr, k, voc, train_features = joblib.load("bof.pkl")
    file=open('bof_p.pkl', 'rb')
    clf, classes_names, stdSlr, k, voc, train_features = pickle.load(file)

    # Create feature extraction and keypoint detector objects
    sift = cv2.xfeatures2d.SIFT_create(200)
    
    # List where all the descriptors are stored
    kpts, descriptors = sift.detectAndCompute(image,None)  
    
    test_features = np.zeros((1, k), "float32")
    
    words, distance = vq(descriptors,voc)
    for w in words:
        test_features[0][w] += 1

    # Perform Tf-Idf vectorization
    nbr_occurences = np.sum( (test_features > 0) * 1, axis = 0)
    idf = np.array(np.log((1.0+1) / (1.0*nbr_occurences + 1)), 'float32')

    # Scale the features
    test_features = stdSlr.transform(test_features)
    prediction =  classes_names[clf.predict(test_features)[0]]
    return prediction

In [ ]:
image_paths = []
testing_names = os.listdir(test_path)
#for testing_name in testing_names:
dir = os.path.join(test_path, "cup")
class_path = imutils.imlist(dir)
image_paths=class_path
print("Accuracy non occluded dataset")
cups = 0;
total_cups = len(image_paths)
for im_path in image_paths:
    im = cv2.imread(im_path)
    p = classifyBoW(im)
    if(p=='cup'):
        cups+=1
    #print("prediction: %s"%(p))
print("Accuracy cups: %g"%(cups/total_cups))


dir = os.path.join(test_path, "book")
class_path = imutils.imlist(dir)
image_paths=class_path
#print(image_paths)

books = 0;
total_books = len(image_paths)
for im_path in image_paths:
    im = cv2.imread(im_path)
    p = classifyBoW(im)
    if(p=='book'):
        books+=1
    #print("prediction: %s"%(p))
print("Accuracy books: %g"%(books/total_books))


dir = os.path.join(test_path, "box")
class_path = imutils.imlist(dir)
image_paths=class_path
#print(image_paths)

boxes = 0;
total_boxes = len(image_paths)
for im_path in image_paths:
    im = cv2.imread(im_path)
    p = classifyBoW(im)
    if(p=='box'):
        boxes+=1
    #print("prediction: %s"%(p))
print("Accuracy boxes: %g"%(boxes/total_boxes))

Accuracy non occluded dataset
Accuracy cups: 0.913462
Accuracy books: 0.803279
Accuracy boxes: 0.953488
